### 数据处理说明
#### 数据来源：2009年微波辐射计亮温数据和探空数据

文件路径中读取日期并转换格式：filepath[69:79].replace('-','/')

In [17]:
import pandas as pd
import os
# 自定义函数
import search_filepth_module

In [18]:
Year = '2009'

In [ ]:
microwave_dirpath = r'I:\Data\Personal Data\graduation project\SACOL\microwave\'+Year
microwave_target_str = 'lv1'
microwave_filepaths = search_filepth_module.search_filepath(microwave_dirpath,microwave_target_str)

In [ ]:
statistics = pd.DataFrame(columns=['Date','lv1_08','lv1_20'])

for microwave_filepath in microwave_filepaths:
    lv1_data = pd.read_csv(microwave_filepath,index_col='Record')
    # lv1_data.drop(['Unnamed: 23'],axis=1,inplace=True)
    print(microwave_filepath)
    print(microwave_filepath[69:79].replace('-','/'),lv1_data.shape)

    num_08 = 0
    num_20 = 0
    for i in lv1_data['Date/Time']:
        if((i[9:14] >= '08:00') and (i[9:14] <= '08:30')):
            num_08 += 1
        if((i[9:14] >= '20:00') and (i[9:14] <= '20:30')):
            num_20 += 1
    
    new = {'Date':microwave_filepath[69:79].replace('-','/'),'lv1_08':num_08,'lv1_20':num_20}
    statistics =statistics.append(new,ignore_index=True)

In [38]:
Wyoming_dirpath = r'I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1'
Wyoming_target_str = Year
Wyoming_filepaths = []
for root,dirs,files in os.walk(Wyoming_dirpath):
    for file in files:
        # print(file[30:34])
        if(Wyoming_target_str == file[30:34]):
            Wyoming_filepath = os.path.join(root,file)
            Wyoming_filepaths.append(Wyoming_filepath)

In [34]:
head_col = ['Press','Altitude','Temperature','Relative_humidity','NaN1','NaN2','NaN3']
Wyoming_data = pd.read_table(Wyoming_filepaths[0],sep='\s+',skiprows=6,names=head_col)
Wyoming_data['Altitude'] = Wyoming_data['Altitude']-1965
Wyoming_data

,Press,Altitude,Temperature,Relative_humidity,NaN1,NaN2,NaN3
0,553.58502,0.0,254.99678,77.53760,-9999.99,-9999.99,-9999.99
1,546.22186,100.0,254.58287,78.46770,-9999.99,-9999.99,-9999.99
2,538.94153,200.0,254.16812,79.39978,-9999.99,-9999.99,-9999.99
3,531.74274,300.0,253.75252,80.33389,-9999.99,-9999.99,-9999.99
4,524.62506,400.0,253.33598,81.26997,-9999.99,-9999.99,-9999.99
5,517.58765,500.0,252.91852,82.20806,-9999.99,-9999.99,-9999.99
6,510.62952,600.0,252.50023,83.14824,-9999.99,-9999.99,-9999.99
7,503.75031,700.0,252.08099,84.09042,-9999.99,-9999.99,-9999.99
8,496.96423,800.0,251.53888,83.96310,-9999.99,-9999.99,-9999.99
9,490.27328,900.0,250.84637,82.52438,-9999.99,-9999.99,-9999.99


In [37]:
print(Wyoming_data.loc[Wyoming_data['Temperature']>-100].Temperature.count())
print(Wyoming_data.loc[Wyoming_data['Relative_humidity']>-100].Relative_humidity.count())

47
42
